## **FasterRCNN Pseudo Labeling**

> The remaing 124 plot images from SVREC 2022 will be used in this notebook to perform the pseudo labeling. 

> The training data is small, therefore more data usually help. 

> The pre-trained fasterRCNN model using the parameters from the tunning process (Hyperparameter search step) was used to perform the pseudo labeling.

> The plant identification was performed using the threshold equal or higher than 0.5.

### Reference notebooks:
- [Train notebook here](https://www.kaggle.com/pestipeti/pytorch-starter-fasterrcnn-train)
- [Inference notebook here](https://www.kaggle.com/pestipeti/pytorch-starter-fasterrcnn-inference)
- [Pseudo Labeling](https://www.kaggle.com/code/nvnnghia/fasterrcnn-pseudo-labeling)



In [87]:
#!pip freeze

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
!pip install timm
!pip install pycocotools

!pip install -q segmentation-models-pytorch
!pip install -q torchsummary

!pip install albumentations==0.4.6

import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN_ResNet50_FPN_Weights

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [90]:
DIR_INPUT = '/content/drive/MyDrive/UAS_Beans/Beans_StandCount/2022/SVREC/ByStep'

DIR_TEST = f'{DIR_INPUT}/test_22_SVREC_RGB_7m_img'

DIR_WEIGHTS = f'{DIR_INPUT}/models/BestModel_wandbHyper'

WEIGHTS_FILE = f'{DIR_WEIGHTS}/fasterrcnn_resnet50_fpn_v01_4bat.pth'

In [91]:
img_list = sorted(os.listdir(f'{DIR_TEST}'))
img_list = [v for v in img_list if v.endswith('.png') and not v.endswith('_msk.png')]
print(img_list)

['2201_3001_7m.png', '2201_3002_7m.png', '2201_3003_7m.png', '2201_3004_7m.png', '2201_3005_7m.png', '2201_3006_7m.png', '2201_3007_7m.png', '2201_3008_7m.png', '2201_3009_7m.png', '2201_3010_7m.png', '2201_3011_7m.png', '2201_3012_7m.png', '2201_3013_7m.png', '2201_3014_7m.png', '2201_3015_7m.png', '2201_3016_7m.png', '2201_3017_7m.png', '2201_3018_7m.png', '2201_3019_7m.png', '2201_3020_7m.png', '2201_3021_7m.png', '2201_3022_7m.png', '2201_3023_7m.png', '2201_3024_7m.png', '2201_3025_7m.png', '2201_3026_7m.png', '2201_3027_7m.png', '2201_3028_7m.png', '2201_3029_7m.png', '2201_3030_7m.png', '2201_3031_7m.png', '2201_3032_7m.png', '2201_3033_7m.png', '2201_3034_7m.png', '2201_3035_7m.png', '2201_3036_7m.png', '2201_4001_7m.png', '2201_4002_7m.png', '2201_4003_7m.png', '2201_4004_7m.png', '2201_4005_7m.png', '2201_4006_7m.png', '2201_4007_7m.png', '2201_4008_7m.png', '2201_4009_7m.png', '2201_4010_7m.png', '2201_4011_7m.png', '2201_4012_7m.png', '2201_4013_7m.png', '2201_4014_7m.png',

In [92]:
img_list_name = []

for img in img_list:
  name = img.replace('.png', '')
  img_list_name.append([name])

#print(img_list_name)

test_df = pd.DataFrame(img_list_name, columns=['image_id'])

test_df["PredictionString"] = np.nan
test_df

,image_id,PredictionString
0,2201_3001_7m,NaN
1,2201_3002_7m,NaN
2,2201_3003_7m,NaN
3,2201_3004_7m,NaN
4,2201_3005_7m,NaN
...,...,...
119,2202_4026_7m,NaN
120,2202_4027_7m,NaN
121,2202_4028_7m,NaN
122,2202_4029_7m,NaN


In [93]:
class BeanTestDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}.png', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']

        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [94]:
# Albumentations
def get_test_transform():
    return A.Compose([
        # A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ])


In [95]:
# load a model; pre-trained on COCO
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False,weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT, box_detections_per_img=180, weights_backbone = None)

In [96]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the trained weights
model.load_state_dict(torch.load(WEIGHTS_FILE))
model.eval()

x = model.to(device)

In [97]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_dataset = BeanTestDataset(test_df, DIR_TEST, get_test_transform())

test_data_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=2,
    drop_last=False,
    collate_fn=collate_fn
)

In [98]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

## **Detection and make Pseudo labels for test dataset**

In [99]:
detection_threshold = 0.5
results = []

testdf_psuedo = []
for images, image_ids in test_data_loader:

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()
        scores = outputs[i]['scores'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        for box in boxes:
            #print(box)
            result = {
                'image_id': image_id,
                'width': 3872,
                'height': 640,
                'source': 'nvnn',
                'x': box[0],
                'y': box[1],
                'w': box[2],
                'h': box[3]
            }
            testdf_psuedo.append(result)
            


In [100]:
test_df_pseudo = pd.DataFrame(testdf_psuedo, columns=['image_id', 'width', 'height', 'source', 'x', 'y', 'w', 'h'])
test_df_pseudo

,image_id,width,height,source,x,y,w,h
0,2201_3001_7m,3872,640,nvnn,1303,469,30,68
1,2201_3001_7m,3872,640,nvnn,2492,496,26,63
2,2201_3001_7m,3872,640,nvnn,1509,487,34,45
3,2201_3001_7m,3872,640,nvnn,899,471,38,38
4,2201_3001_7m,3872,640,nvnn,1019,467,29,53
...,...,...,...,...,...,...,...,...
13638,2202_4030_7m,3872,640,nvnn,2122,141,45,38
13639,2202_4030_7m,3872,640,nvnn,2332,125,38,39
13640,2202_4030_7m,3872,640,nvnn,3745,522,30,41
13641,2202_4030_7m,3872,640,nvnn,998,112,38,41


## **Retrain model with pseudo labels**

In [101]:
train_df = pd.read_csv(f'{DIR_INPUT}/Annot_22_SVREC_RGB_7m/Annot_beans_22_SVREC_7m__3.csv')
train_df['x'] = -1
train_df['y'] = -1
train_df['w'] = -1
train_df['h'] = -1

def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

train_df[['x', 'y', 'w', 'h']] = np.stack(train_df['bbox'].apply(lambda x: expand_bbox(x)))
train_df.drop(columns=['bbox'], inplace=True)
train_df['x'] = train_df['x'].astype(float)
train_df['y'] = train_df['y'].astype(float)
train_df['w'] = train_df['w'].astype(float)
train_df['h'] = train_df['h'].astype(float)

train_df.head()

,image_id,width,height,source,x,y,w,h
0,2202_3008_7m,3872,640,bean,544.903955,135.439642,29.830508,31.475410
1,2202_3008_7m,3872,640,bean,579.706215,124.947839,27.841808,40.059613
2,2202_3008_7m,3872,640,bean,609.536723,141.162444,11.932203,19.076006
3,2202_3008_7m,3872,640,bean,648.316384,123.994039,30.824859,34.336811
4,2202_3008_7m,3872,640,bean,688.090395,146.885246,29.830508,30.521610


In [102]:
frames = [train_df, test_df_pseudo]

train_df = pd.concat(frames)
train_df

,image_id,width,height,source,x,y,w,h
0,2202_3008_7m,3872,640,bean,544.903955,135.439642,29.830508,31.475410
1,2202_3008_7m,3872,640,bean,579.706215,124.947839,27.841808,40.059613
2,2202_3008_7m,3872,640,bean,609.536723,141.162444,11.932203,19.076006
3,2202_3008_7m,3872,640,bean,648.316384,123.994039,30.824859,34.336811
4,2202_3008_7m,3872,640,bean,688.090395,146.885246,29.830508,30.521610
...,...,...,...,...,...,...,...,...
13638,2202_4030_7m,3872,640,nvnn,2122.000000,141.000000,45.000000,38.000000
13639,2202_4030_7m,3872,640,nvnn,2332.000000,125.000000,38.000000,39.000000
13640,2202_4030_7m,3872,640,nvnn,3745.000000,522.000000,30.000000,41.000000
13641,2202_4030_7m,3872,640,nvnn,998.000000,112.000000,38.000000,41.000000


In [103]:
train_df.to_csv(f'{DIR_INPUT}/g._PseudoLab/Annot_beans_22_SVREC_7m_pseudo_annot.csv', index=False)